Adapted from Applied Data Science with Python from Coursera

Here, I clean and join several tables (tables described below.) I convert Energy Supply to gigajoules and return a final table with relevant columns from each table, sorted by the Sciamgo rank. I then use the resultant table for subsequent analyses.
- Energy Indicators.xls, 2013 United Nations data for energy supply/renewable electricity indictors
- world_bank.csv, GDP World Bank data, 1960-2015
- scimagojr-3.xlsx, Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology, ranks countries on journal contributions in renewable electricity/energy supply

In [4]:
import pandas as pd
import numpy as np
energy=pd.read_excel("Energy Indicators.xls", 
                     index_col=None 
                     ,header=None
                     ,skipfooter=1
                    )
energy = energy.drop([0,1], axis=1)
energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
energy.replace(to_replace='...',value=np.NaN, inplace=True)
giga = []
for value in energy['Energy Supply']:
    if type(value) != str and value != np.NaN:
        giga.append(value*1000000.0)
    else:
        giga.append(value)
energy['Energy Supply'] = giga
import re
for value in energy['Country']:
    if type(value) == str and re.match('(.*)[0-9]+$',value):
        match_object = re.search('([A-Za-z, ]+)[0-9]+$',value)
        energy.replace(to_replace=value,value=match_object[1], inplace=True)
energy['Country'].replace({"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"},inplace=True)
for value in energy['Country']:
    if type(value) == str and re.match('[A-Za-z ]*\(.*\)$',value):
        match_object = re.search('([A-Za-z ]*)\(.*\)$',value)
        energy.replace(to_replace=value,value=match_object[1].strip(), inplace=True)
GDP=pd.read_csv("world_bank.csv", 
                     index_col=None 
                     ,header=None
                    )
GDP[0].replace({"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"},inplace=True)
ScimEm=pd.read_excel("scimagojr-3.xlsx", 
                     index_col=None
                    )
country_name = GDP[0]
drop_list = [*range(0,50,1)]
GDP = GDP.drop(drop_list, axis=1)
column_names = []
for value in GDP.iloc[4].values:
    column_names.append(str(int(value)))
GDP.columns = column_names
GDP.insert(0,'Country',country_name.values)
drop_list1 = [*range(15,len(ScimEm),1)]
ScimEm = ScimEm.drop(drop_list1, axis=0)
intermediate = pd.merge(energy, GDP, how='inner', on='Country')
summary = pd.merge(ScimEm, intermediate, how='inner', on='Country').set_index('Country').sort_values(by='Rank', ascending=True)
summary.head(15)

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.27191e+11,93,19.7549,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.0838e+10,286,11.571,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.8984e+10,149,10.2328,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.92e+09,124,10.6005,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.0709e+10,214,17.2887,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.0431e+10,296,61.9454,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.3261e+10,165,17.9015,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.3195e+10,26,14.9691,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.0597e+10,166,17.0203,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [5]:
# Let's return a series that shows the countries sorted by average GDP in descending order.
summary_q3 = summary
for value in summary.columns:
    if not re.match('2.*',value):
        summary_q3.drop(value, axis=1, inplace=True)
x = summary_q3
x['Average GDP'] = summary_q3.mean(axis=1)
avgGDP = pd.Series(x['Average GDP'], index=x.index).sort_values(ascending=False)
print(avgGDP)

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: Average GDP, dtype: float64


In [6]:
# Let's take the country with the 6th highest avg GDP and see how much GDP changed from 2006 to 2015.
x = summary_q3
x['Average GDP'] = x.mean(axis=1)
avgGDP = pd.Series(x['Average GDP'], index=x.index).sort_values(ascending=False)
sixthbest = x.sort_values(by='Average GDP', ascending=False).iloc[5]
print(sixthbest[-2]-sixthbest[0])

246702696075.3999


In [9]:
# Let's calculate avg energy supply per capita.
def avg_energy():
    import pandas as pd
    import numpy as np
    energy=pd.read_excel("Energy Indicators.xls", 
                         index_col=None 
                         ,header=None
                         ,skipfooter=1
                        )
    energy = energy.drop([0,1], axis=1)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.replace(to_replace='...',value=np.NaN, inplace=True)
    giga = []
    for value in energy['Energy Supply']:
        if type(value) != str and value != np.NaN:
            giga.append(value*1000000.0)
        else:
            giga.append(value)
    energy['Energy Supply'] = giga
    import re
    for value in energy['Country']:
        if type(value) == str and re.match('(.*)[0-9]+$',value):
            match_object = re.search('([A-Za-z, ]+)[0-9]+$',value)
            energy.replace(to_replace=value,value=match_object[1], inplace=True)
    energy['Country'].replace({"Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"},inplace=True)
    for value in energy['Country']:
        if type(value) == str and re.match('[A-Za-z ]*\(.*\)$',value):
            match_object = re.search('([A-Za-z ]*)\(.*\)$',value)
            energy.replace(to_replace=value,value=match_object[1].strip(), inplace=True)
    GDP=pd.read_csv("world_bank.csv", 
                         index_col=None 
                         ,header=None
                        )
    GDP[0].replace({"Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"},inplace=True)
    ScimEm=pd.read_excel("scimagojr-3.xlsx", 
                         index_col=None
                        )
    country_name = GDP[0]
    drop_list = [*range(0,50,1)]
    GDP = GDP.drop(drop_list, axis=1)
    column_names = []
    for value in GDP.iloc[4].values:
        column_names.append(str(int(value)))
    GDP.columns = column_names
    GDP.insert(0,'Country',country_name.values)
    drop_list1 = [*range(15,len(ScimEm),1)]
    ScimEm = ScimEm.drop(drop_list1, axis=0)
    intermediate = pd.merge(energy, GDP, how='inner', on='Country')
    summary = pd.merge(ScimEm, intermediate, how='inner', on='Country').set_index('Country').sort_values(by='Rank', ascending=True)
    x = summary.mean(axis=0)
    return x['Energy Supply per Capita']
avg_energy()

157.6

In [11]:
# Let's make a tuple with the name of the country with the highest % Renewable and the percentage.
def max_energy():
    import pandas as pd
    import numpy as np
    energy=pd.read_excel("Energy Indicators.xls", 
                         index_col=None 
                         ,header=None
                         ,skipfooter=1
                        )
    energy = energy.drop([0,1], axis=1)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.replace(to_replace='...',value=np.NaN, inplace=True)
    giga = []
    for value in energy['Energy Supply']:
        if type(value) != str and value != np.NaN:
            giga.append(value*1000000.0)
        else:
            giga.append(value)
    energy['Energy Supply'] = giga
    import re
    for value in energy['Country']:
        if type(value) == str and re.match('(.*)[0-9]+$',value):
            match_object = re.search('([A-Za-z, ]+)[0-9]+$',value)
            energy.replace(to_replace=value,value=match_object[1], inplace=True)
    energy['Country'].replace({"Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"},inplace=True)
    for value in energy['Country']:
        if type(value) == str and re.match('[A-Za-z ]*\(.*\)$',value):
            match_object = re.search('([A-Za-z ]*)\(.*\)$',value)
            energy.replace(to_replace=value,value=match_object[1].strip(), inplace=True)
    GDP=pd.read_csv("world_bank.csv", 
                         index_col=None 
                         ,header=None
                        )
    GDP[0].replace({"Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"},inplace=True)
    ScimEm=pd.read_excel("scimagojr-3.xlsx", 
                         index_col=None
                        )
    country_name = GDP[0]
    drop_list = [*range(0,50,1)]
    GDP = GDP.drop(drop_list, axis=1)
    column_names = []
    for value in GDP.iloc[4].values:
        column_names.append(str(int(value)))
    GDP.columns = column_names
    GDP.insert(0,'Country',country_name.values)
    drop_list1 = [*range(15,len(ScimEm),1)]
    ScimEm = ScimEm.drop(drop_list1, axis=0)
    intermediate = pd.merge(energy, GDP, how='inner', on='Country')
    summary = pd.merge(ScimEm, intermediate, how='inner', on='Country').set_index('Country').sort_values(by='Rank', ascending=True)
    q5 = summary.sort_values(by='% Renewable', ascending=False)
    return ((q5.iloc[0].name, q5.iloc[0]['% Renewable']))
max_energy()

('Brazil', 69.64803)

In [12]:
# Here, we estimate population with the Energy Supply per Capita and Energy Supply columns. 
# Using this, I make a new column for citable documents per capita. Then, I return the correlation
# between energy supply per capita and citable documents per capita.
def doc_energy_corr():
    import pandas as pd
    import numpy as np
    energy=pd.read_excel("Energy Indicators.xls", 
                         index_col=None 
                         ,header=None
                         ,skipfooter=1
                        )
    energy = energy.drop([0,1], axis=1)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.replace(to_replace='...',value=np.NaN, inplace=True)
    giga = []
    for value in energy['Energy Supply']:
        if type(value) != str and value != np.NaN:
            giga.append(value*1000000.0)
        else:
            giga.append(value)
    energy['Energy Supply'] = giga
    import re
    for value in energy['Country']:
        if type(value) == str and re.match('(.*)[0-9]+$',value):
            match_object = re.search('([A-Za-z, ]+)[0-9]+$',value)
            energy.replace(to_replace=value,value=match_object[1], inplace=True)
    energy['Country'].replace({"Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"},inplace=True)
    for value in energy['Country']:
        if type(value) == str and re.match('[A-Za-z ]*\(.*\)$',value):
            match_object = re.search('([A-Za-z ]*)\(.*\)$',value)
            energy.replace(to_replace=value,value=match_object[1].strip(), inplace=True)
    GDP=pd.read_csv("world_bank.csv", 
                         index_col=None 
                         ,header=None
                        )
    GDP[0].replace({"Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"},inplace=True)
    ScimEm=pd.read_excel("scimagojr-3.xlsx", 
                         index_col=None
                        )
    country_name = GDP[0]
    drop_list = [*range(0,50,1)]
    GDP = GDP.drop(drop_list, axis=1)
    column_names = []
    for value in GDP.iloc[4].values:
        column_names.append(str(int(value)))
    GDP.columns = column_names
    GDP.insert(0,'Country',country_name.values)
    drop_list1 = [*range(15,len(ScimEm),1)]
    ScimEm = ScimEm.drop(drop_list1, axis=0)
    intermediate = pd.merge(energy, GDP, how='inner', on='Country')
    summary = pd.merge(ScimEm, intermediate, how='inner', on='Country').set_index('Country').sort_values(by='Rank', ascending=True)
    list1 = []
    list2 = []
    list3 = []
    for value1 in summary['Energy Supply']:
        list1.append(value1)
    for value2 in summary['Energy Supply per Capita']:
        list2.append(value2)
    for i in range(len(list1)):
        list3.append(list1[i]/list2[i])
    q8 = summary
    q8['Pop Estimate'] = list3
    q9 = q8
    list4 = []
    list5 = []
    for value1 in summary['Citable documents']:
        list4.append(value1)
    for i in range(len(list1)):
        list5.append(list4[i]/list3[i])
    q9['Docs per person Estimate'] = list5
    import scipy.stats as stats
    corr, pval=stats.pearsonr(q9['Energy Supply per Capita'],q9['Docs per person Estimate'])
    return corr
doc_energy_corr()

0.7940010435442943

In [13]:
# Here, we add continent bins for each country. We use dataframe.groupby().agg() to make a new dataframe with
# the mean, sum, and stddev of each country's population and how many countries there are for each continent bin.
def continents():
    import pandas as pd
    import numpy as np
    energy=pd.read_excel("Energy Indicators.xls", 
                         index_col=None 
                         ,header=None
                         ,skipfooter=1
                        )
    energy = energy.drop([0,1], axis=1)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.replace(to_replace='...',value=np.NaN, inplace=True)
    giga = []
    for value in energy['Energy Supply']:
        if type(value) != str and value != np.NaN:
            giga.append(value*1000000.0)
        else:
            giga.append(value)
    energy['Energy Supply'] = giga
    import re
    for value in energy['Country']:
        if type(value) == str and re.match('(.*)[0-9]+$',value):
            match_object = re.search('([A-Za-z, ]+)[0-9]+$',value)
            energy.replace(to_replace=value,value=match_object[1], inplace=True)
    energy['Country'].replace({"Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"},inplace=True)
    for value in energy['Country']:
        if type(value) == str and re.match('[A-Za-z ]*\(.*\)$',value):
            match_object = re.search('([A-Za-z ]*)\(.*\)$',value)
            energy.replace(to_replace=value,value=match_object[1].strip(), inplace=True)
    GDP=pd.read_csv("world_bank.csv", 
                         index_col=None 
                         ,header=None
                        )
    GDP[0].replace({"Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"},inplace=True)
    ScimEm=pd.read_excel("scimagojr-3.xlsx", 
                         index_col=None
                        )
    country_name = GDP[0]
    drop_list = [*range(0,50,1)]
    GDP = GDP.drop(drop_list, axis=1)
    column_names = []
    for value in GDP.iloc[4].values:
        column_names.append(str(int(value)))
    GDP.columns = column_names
    GDP.insert(0,'Country',country_name.values)
    drop_list1 = [*range(15,len(ScimEm),1)]
    ScimEm = ScimEm.drop(drop_list1, axis=0)
    intermediate = pd.merge(energy, GDP, how='inner', on='Country')
    summary = pd.merge(ScimEm, intermediate, how='inner', on='Country').set_index('Country').sort_values(by='Rank', ascending=True)
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    list1 = []
    list2 = []
    list3 = []
    for value1 in summary['Energy Supply']:
        list1.append(value1)
    for value2 in summary['Energy Supply per Capita']:
        list2.append(value2)
    for i in range(len(list1)):
        list3.append(list1[i]/list2[i])
    summary['Popn Estimate'] = list3
    list4 = []
    for value in summary.index:
        list4.append(ContinentDict[value])
    summary['Bin'] = list4
    summary = summary.reset_index()
    summary = summary.groupby("Bin").agg({"Popn Estimate":(np.nanmean,np.nanstd,np.nansum),"Country":np.count_nonzero})
    summary.head(5)
    summary = summary.reset_index()
    summary.columns = ['Continent','mean', 'std','sum','size']
    summary.set_index('Continent',inplace=True)
    summary = summary[['size','sum','mean','std']]
    return summary
continents()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [14]:
# Lastly, we do some string slicing. I add our population estimate column to the summary dataframe again
# and then convert each value to a string, with the same number of significant figures and with commas added
# to separate thousands.
def string_slicing():
    import pandas as pd
    import numpy as np
    energy=pd.read_excel("Energy Indicators.xls", 
                         index_col=None 
                         ,header=None
                         ,skipfooter=1
                        )
    energy = energy.drop([0,1], axis=1)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.replace(to_replace='...',value=np.NaN, inplace=True)
    giga = []
    for value in energy['Energy Supply']:
        if type(value) != str and value != np.NaN:
            giga.append(value*1000000.0)
        else:
            giga.append(value)
    energy['Energy Supply'] = giga
    import re
    for value in energy['Country']:
        if type(value) == str and re.match('(.*)[0-9]+$',value):
            match_object = re.search('([A-Za-z, ]+)[0-9]+$',value)
            energy.replace(to_replace=value,value=match_object[1], inplace=True)
    energy['Country'].replace({"Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"},inplace=True)
    for value in energy['Country']:
        if type(value) == str and re.match('[A-Za-z ]*\(.*\)$',value):
            match_object = re.search('([A-Za-z ]*)\(.*\)$',value)
            energy.replace(to_replace=value,value=match_object[1].strip(), inplace=True)
    GDP=pd.read_csv("world_bank.csv", 
                         index_col=None 
                         ,header=None
                        )
    GDP[0].replace({"Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"},inplace=True)
    ScimEm=pd.read_excel("scimagojr-3.xlsx", 
                         index_col=None
                        )
    country_name = GDP[0]
    drop_list = [*range(0,50,1)]
    GDP = GDP.drop(drop_list, axis=1)
    column_names = []
    for value in GDP.iloc[4].values:
        column_names.append(str(int(value)))
    GDP.columns = column_names
    GDP.insert(0,'Country',country_name.values)
    drop_list1 = [*range(15,len(ScimEm),1)]
    ScimEm = ScimEm.drop(drop_list1, axis=0)
    intermediate = pd.merge(energy, GDP, how='inner', on='Country')
    summary = pd.merge(ScimEm, intermediate, how='inner', on='Country').set_index('Country').sort_values(by='Rank', ascending=True)
    list1 = []
    list2 = []
    list3 = []
    for value1 in summary['Energy Supply']:
        list1.append(value1)
    for value2 in summary['Energy Supply per Capita']:
        list2.append(value2)
    for i in range(len(list1)):
        list3.append(list1[i]/list2[i])
    summary['Pop Estimate'] = list3
    q13 = summary['Pop Estimate']
    list4 = []
    for value in q13.values:
        list4.append(str(value))
    import re
    before_decimal = []
    after_decimal = []
    before_decimal_with_commas = []
    for value in list4:
        x,y=value.split('.')
        before_decimal.append(x)
        after_decimal.append(y)
    for value in before_decimal:
        string_holder = ''
        string_holder2 = ''
        counter = 0
        for i in value[::-1]:
            if counter%3 == 0 and counter != 0:
                string_holder += ','
            string_holder += (i)
            counter += 1
        for value2 in string_holder[::-1]:
            string_holder2 += value2
        before_decimal_with_commas.append(string_holder2)
    counter1 = 0
    final_list = []
    final_holder = ''
    while counter1 != len(after_decimal):
        final_list.append((before_decimal_with_commas[counter1]+'.'+after_decimal[counter1]))
        counter1 += 1
    summary['Pop Estimate'] = final_list
    q13 = summary['Pop Estimate']
    return q13
string_slicing()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: Pop Estimate, dtype: object